In [49]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from skimage import io
from shutil import copyfile
import sys
import time

In [7]:
# Download the files from https://storage.googleapis.com/openimages/web/download.html
images_fids = './Data/Source_Images/OIDsample/train/image_ids_and_rotation.csv'
annotations_bbox_fids = './Data/Source_Images/OIDsample/train/oidv6-train-annotations-bbox.csv'
class_descriptions_fids = './Data/Source_Images/OIDsample/train/class-descriptions-boxable.csv'

In [8]:
images_info = pd.read_csv(images_fids)
images_info.head()

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,4fa8054781a4c382,train,https://farm3.staticflickr.com/5310/5898076654...,https://www.flickr.com/photos/michael-beat/589...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/michael-beat/,Michael Beat,...die FNF-Kerze,4405052,KFukvivpCM5QXl5SqKe41g==,https://c1.staticflickr.com/6/5310/5898076654_...,0.0
1,d05c3e451f79174d,train,https://c6.staticflickr.com/8/7375/13791738583...,https://www.flickr.com/photos/jeepersmedia/137...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/jeepersmedia/,Mike Mozart,Pez Kids Breakfast Cereal Prizes,972558,KoVMrVWKftzadg1EGr6nCg==,https://c1.staticflickr.com/8/7375/13791738583...,0.0
2,5bde9ad21e82deed,train,https://c6.staticflickr.com/1/223/495658260_89...,https://www.flickr.com/photos/8106459@N07/4956...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/8106459@N07/,David~O,Isabella Lake,234199,dsfj7rNAbGq67OzPRAGdsQ==,https://c2.staticflickr.com/1/223/495658260_85...,0.0
3,b37f763ae67d0888,train,https://c1.staticflickr.com/1/67/197493648_628...,https://www.flickr.com/photos/drstarbuck/19749...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/drstarbuck/,Karen,Three boys on a hill,494555,9IzEn38GRNsVpATuv7gzEA==,https://c3.staticflickr.com/1/67/197493648_628...,0.0
4,3231e75bd7120b7a,train,https://c3.staticflickr.com/4/3654/3619297168_...,https://www.flickr.com/photos/cckaiser/3619297168,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/cckaiser/,Colin,DSC_0096,2570210,195AcCHMqin3MYTv+pTDuA==,https://c4.staticflickr.com/4/3654/3619297168_...,0.0


In [9]:
annotations_bbox = pd.read_csv(annotations_bbox_fids)
annotations_bbox.head() 

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,...,IsDepiction,IsInside,XClick1X,XClick2X,XClick3X,XClick4X,XClick1Y,XClick2Y,XClick3Y,XClick4Y
0,000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,...,0,0,0.148438,0.012500,0.059375,0.195312,0.148438,0.357812,0.587500,0.325000
1,000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,...,0,0,0.025000,0.248438,0.276563,0.214062,0.914062,0.714063,0.782813,0.948438
2,000002b66c9c498e,xclick,/m/01g317,1,0.151562,0.310937,0.198437,0.590625,1,0,...,0,0,0.243750,0.151562,0.310937,0.262500,0.198437,0.434375,0.507812,0.590625
3,000002b66c9c498e,xclick,/m/01g317,1,0.256250,0.429688,0.651563,0.925000,1,0,...,0,0,0.315625,0.429688,0.256250,0.423438,0.651563,0.921875,0.826562,0.925000
4,000002b66c9c498e,xclick,/m/01g317,1,0.257812,0.346875,0.235938,0.385938,1,0,...,0,0,0.317188,0.257812,0.346875,0.307812,0.235938,0.289062,0.348438,0.385938


In [10]:
class_descriptions=pd.read_csv(class_descriptions_fids, header=None)
class_descriptions.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [11]:
class_descriptions.loc[class_descriptions[1].isin(['Person', 'Car', 'Human face', 'Laptop','Mobile phone'  ])]

,0,1
54,/m/01c648,Laptop
68,/m/01g317,Person
312,/m/050k8,Mobile phone
501,/m/0dzct,Human face
570,/m/0k4j,Car


In [12]:
person_pd = class_descriptions[class_descriptions[1]=='Person']
face_pd = class_descriptions[class_descriptions[1]=='Human face']
car_pd = class_descriptions[class_descriptions[1]=='Car']
mobile_pd = class_descriptions[class_descriptions[1]=='Mobile phone']
laptop_pd = class_descriptions[class_descriptions[1]=='Laptop']

label_name_person = person_pd[0].values[0]
label_name_face = face_pd[0].values[0]
label_name_car = car_pd[0].values[0]
label_name_mobile = mobile_pd[0].values[0]
label_name_laptop = laptop_pd[0].values[0]


In [13]:
#Bounding of annotation of each of the mentioned classess
person_bbox = annotations_bbox[annotations_bbox['LabelName']==label_name_person]
face_bbox = annotations_bbox[annotations_bbox['LabelName']==label_name_face]
car_bbox = annotations_bbox[annotations_bbox['LabelName']==label_name_car]
laptop_bbox = annotations_bbox[annotations_bbox['LabelName']==label_name_laptop]
mobile_bbox = annotations_bbox[annotations_bbox['LabelName']==label_name_mobile]

In [14]:
person_bbox

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,...,IsDepiction,IsInside,XClick1X,XClick2X,XClick3X,XClick4X,XClick1Y,XClick2Y,XClick3Y,XClick4Y
0,000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,...,0,0,0.148438,0.012500,0.059375,0.195312,0.148438,0.357812,0.587500,0.325000
1,000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,...,0,0,0.025000,0.248438,0.276563,0.214062,0.914062,0.714063,0.782813,0.948438
2,000002b66c9c498e,xclick,/m/01g317,1,0.151562,0.310937,0.198437,0.590625,1,0,...,0,0,0.243750,0.151562,0.310937,0.262500,0.198437,0.434375,0.507812,0.590625
3,000002b66c9c498e,xclick,/m/01g317,1,0.256250,0.429688,0.651563,0.925000,1,0,...,0,0,0.315625,0.429688,0.256250,0.423438,0.651563,0.921875,0.826562,0.925000
4,000002b66c9c498e,xclick,/m/01g317,1,0.257812,0.346875,0.235938,0.385938,1,0,...,0,0,0.317188,0.257812,0.346875,0.307812,0.235938,0.289062,0.348438,0.385938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14610053,ffff22d9d07ff188,xclick,/m/01g317,1,0.625000,0.836250,0.185000,0.998333,1,1,...,1,0,0.705000,0.625000,0.637500,0.836250,0.185000,0.535000,0.998333,0.446667
14610054,ffff22d9d07ff188,xclick,/m/01g317,1,0.780000,0.998750,0.180000,0.996667,1,1,...,1,0,0.865000,0.780000,0.982500,0.998750,0.180000,0.568333,0.996667,0.405000
14610084,ffff59df073e186d,xclick,/m/01g317,1,0.026144,0.160131,0.704248,0.918301,1,0,...,1,0,0.071895,0.026144,0.094771,0.160131,0.704248,0.906863,0.918301,0.892157
14610135,ffff83c6a0e1c6c0,xclick,/m/01g317,1,0.689375,0.960625,0.119167,0.645000,0,0,...,0,0,0.689375,0.883125,0.960625,0.868750,0.310833,0.119167,0.336667,0.645000


In [15]:
person_img_id = person_bbox['ImageID']
face_img_id = face_bbox['ImageID']
car_img_id = car_bbox['ImageID']
laptop_img_id = laptop_bbox['ImageID']
mobile_img_id = mobile_bbox['ImageID']

In [42]:
# Randomly sample n images for downloading. Here, I am selecting 1000 images per class.
n = 10
subperson_id = random.sample(list(person_img_id), n)
subface_id = random.sample(list(face_img_id), n)
subcar_id = random.sample(list(car_img_id), n)
submobile_id = random.sample(list(mobile_img_id), n)
sublaptop_id = random.sample(list(laptop_img_id), n)



In [43]:
subperson_pd = images_boxable.loc[images_boxable['ImageID'].isin(subperson_id)]
subface_pd = images_boxable.loc[images_boxable['ImageID'].isin(subface_id)]
subcar_pd = images_boxable.loc[images_boxable['ImageID'].isin(subcar_id)]
submobile_pd = images_boxable.loc[images_boxable['ImageID'].isin(submobile_id)]
sublaptop_pd = images_boxable.loc[images_boxable['ImageID'].isin(sublaptop_id)]

In [44]:
subperson_pd

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
1728658,cd29f90a6eb21256,train,https://farm7.staticflickr.com/2558/3734522514...,https://www.flickr.com/photos/bmh4you/3734522514,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/bmh4you/,b r e n t,shaved ice and fruit from Guppy House; this is...,1601934,vw33hVHlyhIFW0yvG8tU8A==,https://c4.staticflickr.com/3/2558/3734522514_...,0.0
2369321,628bd545226c2346,train,https://farm7.staticflickr.com/7346/9071181824...,https://www.flickr.com/photos/fortrucker/90711...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/fortrucker/,Fort Rucker,Army Strong Triathlon,3322053,i0MFGv3xnPQY7gbigubztw==,https://c2.staticflickr.com/8/7346/9071181824_...,0.0
2442100,dc9d070e1d815d87,train,https://farm8.staticflickr.com/5258/5526105924...,https://www.flickr.com/photos/fukapon/5526105924,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/fukapon/,Ryo FUKAsawa,ソフマップ 秋葉原 PCゲーム・アニメ館 店頭広告 穢翼のユースティア デモの代わりに「マス...,6598198,ATCuGPM3SWWa/WiBQCut2Q==,https://c1.staticflickr.com/6/5258/5526105924_...,0.0
4360305,0d69a179c4cd8cd0,train,https://farm1.staticflickr.com/8645/1518902718...,https://www.flickr.com/photos/dawn-pinkchick/1...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/dawn-pinkchick/,Dawn,2014Nov-LastChanceHalf 820,7719880,CXqW0aI0Ifh0U0ZevSSu4g==,https://c2.staticflickr.com/9/8645/15189027183...,0.0
6475030,6dca790ea8897cee,train,https://c7.staticflickr.com/9/8335/8096885321_...,https://www.flickr.com/photos/nseika/8096885321,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/nseika/,Seika,P1070922_DxO,738906,DuIH7/HGMs43WlBR260yFg==,https://c8.staticflickr.com/9/8335/8096885321_...,0.0
6525411,b3a3ef1b53f5e220,train,https://farm1.staticflickr.com/3342/3454473314...,https://www.flickr.com/photos/greentechmedia/3...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/greentechmedia/,Greentech Media,Surviving the Shakeout,321793,pcpQ6wj1SjQf1jDOzSiNag==,https://c4.staticflickr.com/4/3342/3454473314_...,0.0
7274984,92df34abd810a081,train,https://farm2.staticflickr.com/5288/5373026908...,https://www.flickr.com/photos/mpmg/5373026908,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/mpmg/,Ministerio Público de Minas Gerais AssCom,IMG_6346,446924,ErGQrPHk8tIwuwu4CqfnOg==,https://c5.staticflickr.com/6/5288/5373026908_...,0.0
7781305,9ee656c101f845e5,train,https://farm6.staticflickr.com/3598/3510468087...,https://www.flickr.com/photos/comunicati/35104...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/comunicati/,Michela Simoncini,Koninginnedag,4081870,wIq1P2NqCSqDNM/b3JVecA==,https://c6.staticflickr.com/4/3598/3510468087_...,0.0
8224225,23b2f1938ebfb781,train,https://farm2.staticflickr.com/3094/2801580588...,https://www.flickr.com/photos/srgblog/2801580588,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/srgblog/,srgpicker,Gente,2759907,h63WDD7ABtRWak98EsoRwQ==,https://c6.staticflickr.com/4/3094/2801580588_...,0.0
8862042,bd33f7ac0f5b7cb8,train,https://c1.staticflickr.com/6/5283/5374766333_...,https://www.flickr.com/photos/ingermaaike2/537...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/ingermaaike2/,inger maaike,rustic leather copper vest,1800961,G+S1ZrQ771ZEaTFmPAa1sg==,https://c2.staticflickr.com/6/5283/5374766333_...,0.0


In [45]:
# link to download the selected sample images
subperson_dict = subperson_pd[["ImageID", "OriginalURL"]].set_index('ImageID')["OriginalURL"].to_dict()
subface_dict = subface_pd[["ImageID", "OriginalURL"]].set_index('ImageID')["OriginalURL"].to_dict()
subcar_dict = subcar_pd[["ImageID", "OriginalURL"]].set_index('ImageID')["OriginalURL"].to_dict()
sublaptop_dict = sublaptop_pd[["ImageID", "OriginalURL"]].set_index('ImageID')["OriginalURL"].to_dict()
submobile_dict = submobile_pd[["ImageID", "OriginalURL"]].set_index('ImageID')["OriginalURL"].to_dict()

In [46]:
lis = [subperson_dict, subface_dict, subcar_dict, submobile_dict, sublaptop_dict]

In [50]:
classes = ['Person', 'Human face', 'Car', 'Mobile phone','Laptop']
for idx, cls in enumerate(classes):
    print(idx,cls)
    for img_id, url in lis[idx].items():
        print(img_id)

0 Person
cd29f90a6eb21256
628bd545226c2346
dc9d070e1d815d87
0d69a179c4cd8cd0
6dca790ea8897cee
b3a3ef1b53f5e220
92df34abd810a081
9ee656c101f845e5
23b2f1938ebfb781
bd33f7ac0f5b7cb8
1 Human face
97d18b64771fa050
0000c7640c802faf
b9cf0b76b457aeaf
465102cd481dd4a6
8351ef2ef7e924a6
1c41e2ed3b52a63d
087caaa8ae25daa6
7e7e721915ec4adc
894c9e590d4835f6
27c85106b82b7e41
2 Car
0bc54c8ee7b3eb5a
4cd5aa31da6fb12c
aeba928533e40cd4
bd879c6005ddefbf
494e083431ef5f91
604f5106d43765b8
0004d295cfb46842
3bbf15b49c905b84
fa1b032073a0dc26
5d5aaf43cfce8afe
3 Mobile phone
604df1825366c00d
814a5a67b7a572b8
462dac5e1d0736ff
3d572b23d1a02288
55e89624eb04734e
a34343489f96c1f3
dea6f6bd59cad265
ebff52584797a689
0efdc9211c56ad92
529a0b96ad053ca1
4 Laptop
302c446b411d8036
0a11d9557cced2dc
da95aceea96be92a
49113b85a7f4a325
43d1ab872f4b5b9c
0e2d81f407ee8196
a33ccc8dea8382db
f67db5ff6cd89a9e
1915cb8fb343474c
c9754b4f90546455


In [57]:
# Now download images

for idx, cls in enumerate(classes):
  dire = './Data/Train/'+cls
  if not os.path.exists(dire):
    os.mkdir(dire)
  for img_id, url in lis[idx].items():
    try:
      img = io.imread(url)
      saved_path = os.path.join(dire, img_id+".jpg")
      io.imsave(saved_path, img)
    except Exception as e:
      print('handled')

handled
handled
handled
handled
handled
handled


In [58]:
# save images to train and test directory
train_path = './Data/Train/'
test_path = './Data/Test/'

In [59]:
# copy downloaded images of each of the folder to a common train folder
split = int(n*0.75)  # spliting train and test
for i in range(len(classes)):
    all_imgs = os.listdir(classes[i])
    all_imgs = [f for f in all_imgs if not f.startswith('.')]
    random.shuffle(all_imgs)
    train_imgs = all_imgs[:split]
    test_imgs = all_imgs[split:]
    
    # copy each classes' images to train directory
    for j in range(len(train_imgs)):
        original_path = os.path.join(classes[i], train_imgs[j])
        new_path = os.path.join(train_path, train_imgs[j])
        copyfile(original_path, new_path)
    
    # copy each classes' images to test directory
    for j in range(len(test_imgs)):
        original_path = os.path.join(classes[i], test_imgs[j])
        new_path = os.path.join(test_path, test_imgs[j])
        copyfile(original_path, new_path)

In [61]:
# create training data: Check the labels below
label_names = [label_name_person, label_name_face,label_name_car, label_name_mobile, label_name_laptop]

train_df = pd.DataFrame(columns=['FileName', 'XMin', 'YMin', 'XMax', 'YMax', 'ClassName'])

# Find boxes in each image and put them in a dataframe
train_imgs = os.listdir(train_path)
train_imgs = [name for name in train_imgs if not name.startswith('.')]

for i in range(len(train_imgs)):
    sys.stdout.write('Parse train_imgs ' + str(i) + '; Number of boxes: ' + str(len(train_df)) + '\r')
    sys.stdout.flush()
    img_name = train_imgs[i]
    img_id = img_name[0:16]
    tmp_df = annotations_bbox[annotations_bbox['ImageID']==img_id]
    for index, row in tmp_df.iterrows():
        labelName = row['LabelName']
        for i in range(len(label_names)):
            if labelName == label_names[i]:
                train_df = train_df.append({'FileName': img_name, 
                                            'XMin': row['XMin'], 
                                            'YMin': row['YMin'], 
                                            'XMax': row['XMax'], 
                                            'YMax': row['YMax'], 
                                            'ClassName': classes[i]}, 
                                           ignore_index=True)

In [62]:
train_img_ids = train_df["FileName"].head().str.split(".").str[0].unique()

In [63]:
# create csv of the dataframe
train_df.to_csv('train.csv')

In [70]:
# Create Dictionary of class names
classes_dict = {'Person':0, 'Human face':1, 'Car':2, 'Mobile phone':3,'Laptop':4}

In [71]:
# create the annotation file for training
train_df = pd.read_csv('train.csv')

# for training
with open("trainannotation.txt", "w+") as f:
  for idx, row in train_df.iterrows():
      img = cv2.imread(train_path + row['FileName'])
      height, width = img.shape[:2]
      x1 = int(row['XMin'] * width)
      y1 = int(row['YMin'] * height)
      x2 = int(row['XMax'] * width)
      y2 = int(row['YMax'] * height)
      
      file_path = 'train'
      fileName = os.path.join(train_path, row['FileName'])
      className = row['ClassName']
      print(fileName + ' ' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + str(classes_dict[className]))
      f.write(fileName + ' ' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + str(classes_dict[className]) + '\n')

./Data/Train/0000c7640c802faf.jpg 541,634,765,1061,1
./Data/Train/0000c7640c802faf.jpg 1077,460,1250,899,1
./Data/Train/0000c7640c802faf.jpg 1201,521,1535,972,1
./Data/Train/0000c7640c802faf.jpg 1701,314,2152,957,1
./Data/Train/087caaa8ae25daa6.jpg 141,134,277,324,1
./Data/Train/087caaa8ae25daa6.jpg 374,86,572,350,1
./Data/Train/0a11d9557cced2dc.jpg 823,857,1160,1149,4
./Data/Train/0a11d9557cced2dc.jpg 526,657,745,738,4
./Data/Train/0bc54c8ee7b3eb5a.jpg 55,8,1255,720,2
./Data/Train/0bc54c8ee7b3eb5a.jpg 818,75,828,92,2
./Data/Train/0bc54c8ee7b3eb5a.jpg 823,78,835,99,2
./Data/Train/0bc54c8ee7b3eb5a.jpg 831,74,863,102,2
./Data/Train/0bc54c8ee7b3eb5a.jpg 845,78,872,104,2
./Data/Train/0bc54c8ee7b3eb5a.jpg 863,78,901,114,2
./Data/Train/0bc54c8ee7b3eb5a.jpg 872,86,927,114,2
./Data/Train/0e2d81f407ee8196.jpg 3942,2775,5924,4612,4
./Data/Train/0e2d81f407ee8196.jpg 6739,2601,7249,2725,4
./Data/Train/0efdc9211c56ad92.jpg 0,16,2541,3254,3
./Data/Train/1915cb8fb343474c.jpg 519,1967,1286,2425,4
./Da

./Data/Train/23b2f1938ebfb781.jpg 1445,803,1511,976,0
./Data/Train/23b2f1938ebfb781.jpg 1453,287,1485,329,0
./Data/Train/23b2f1938ebfb781.jpg 1466,581,1500,701,0
./Data/Train/23b2f1938ebfb781.jpg 1477,1380,1562,1636,0
./Data/Train/23b2f1938ebfb781.jpg 1481,203,1543,280,0
./Data/Train/23b2f1938ebfb781.jpg 1485,148,1504,184,0
./Data/Train/23b2f1938ebfb781.jpg 1485,413,1511,460,0
./Data/Train/23b2f1938ebfb781.jpg 1485,284,1519,338,0
./Data/Train/23b2f1938ebfb781.jpg 1485,295,1705,487,0
./Data/Train/23b2f1938ebfb781.jpg 1488,342,1511,412,0
./Data/Train/23b2f1938ebfb781.jpg 1488,459,1530,566,0
./Data/Train/23b2f1938ebfb781.jpg 1515,342,1537,408,0
./Data/Train/23b2f1938ebfb781.jpg 1515,229,1543,280,0
./Data/Train/23b2f1938ebfb781.jpg 1515,810,1577,976,0
./Data/Train/23b2f1938ebfb781.jpg 1522,366,1562,427,0
./Data/Train/23b2f1938ebfb781.jpg 1526,141,1547,171,0
./Data/Train/23b2f1938ebfb781.jpg 1530,295,1566,329,0
./Data/Train/23b2f1938ebfb781.jpg 1534,203,1562,248,0
./Data/Train/23b2f1938ebfb

./Data/Train/23b2f1938ebfb781.jpg 2517,643,2690,1098,0
./Data/Train/23b2f1938ebfb781.jpg 2521,833,2590,968,0
./Data/Train/23b2f1938ebfb781.jpg 2524,756,2575,888,0
./Data/Train/23b2f1938ebfb781.jpg 2528,910,2590,1104,0
./Data/Train/23b2f1938ebfb781.jpg 2549,748,2609,854,0
./Data/Train/23b2f1938ebfb781.jpg 2575,810,2633,918,0
./Data/Train/23b2f1938ebfb781.jpg 2583,906,2641,1061,0
./Data/Train/23b2f1938ebfb781.jpg 2609,799,2652,910,0
./Data/Train/23b2f1938ebfb781.jpg 2630,641,2690,760,0
./Data/Train/23b2f1938ebfb781.jpg 2633,1467,2752,1725,0
./Data/Train/23b2f1938ebfb781.jpg 2641,696,2690,825,0
./Data/Train/23b2f1938ebfb781.jpg 2648,831,2748,997,0
./Data/Train/23b2f1938ebfb781.jpg 2675,656,2737,799,0
./Data/Train/23b2f1938ebfb781.jpg 2699,788,2759,918,0
./Data/Train/23b2f1938ebfb781.jpg 2710,809,2803,955,0
./Data/Train/27c85106b82b7e41.jpg 424,93,529,238,1
./Data/Train/302c446b411d8036.jpg 470,1303,2082,2299,4
./Data/Train/302c446b411d8036.jpg 1254,389,2928,2286,0
./Data/Train/302c446b411

./Data/Train/dc9d070e1d815d87.jpg 1626,2230,2298,3892,0
./Data/Train/dc9d070e1d815d87.jpg 1646,2519,2111,3141,0
./Data/Train/dc9d070e1d815d87.jpg 2479,2216,3051,4669,0
./Data/Train/dc9d070e1d815d87.jpg 2806,2409,3101,4669,0
./Data/Train/dc9d070e1d815d87.jpg 478,2455,779,2829,1
./Data/Train/dc9d070e1d815d87.jpg 1077,1363,1220,1603,1
./Data/Train/dc9d070e1d815d87.jpg 1220,1722,1372,1991,1
./Data/Train/dc9d070e1d815d87.jpg 1772,2768,2026,3109,1
./Data/Train/dc9d070e1d815d87.jpg 2035,1833,2219,1968,1
./Data/Train/dc9d070e1d815d87.jpg 2625,2441,2736,2838,1
./Data/Train/dc9d070e1d815d87.jpg 2864,2654,3007,2990,1
./Data/Train/ebff52584797a689.jpg 1306,816,1835,1906,3
./Data/Train/fa1b032073a0dc26.jpg 222,406,3512,2309,2
./Data/Train/fa1b032073a0dc26.jpg 1513,376,2004,543,2
./Data/Train/fa1b032073a0dc26.jpg 3292,352,3908,1184,2
